In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv("data/raw.githubusercontent.com_alexeygrigorev_datasets_master_housing.csv")

In [3]:
df

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY
...,...,...,...,...,...,...,...,...,...,...
20635,-121.09,39.48,25.0,1665.0,374.0,845.0,330.0,1.5603,78100.0,INLAND
20636,-121.21,39.49,18.0,697.0,150.0,356.0,114.0,2.5568,77100.0,INLAND
20637,-121.22,39.43,17.0,2254.0,485.0,1007.0,433.0,1.7000,92300.0,INLAND
20638,-121.32,39.43,18.0,1860.0,409.0,741.0,349.0,1.8672,84700.0,INLAND


In [4]:
fil=['<1H OCEAN','INLAND']

In [5]:
df=df[df.ocean_proximity.isin(fil)]

In [6]:
fil_columns=[
'latitude',
'longitude',
'housing_median_age',
'total_rooms',
'total_bedrooms',
'population',
'households',
'median_income',
'median_house_value']


In [7]:
df=df[fil_columns]

In [61]:
df

,latitude,longitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
701,37.64,-121.97,32.0,1283.0,194.0,485.0,171.0,6.0574,431000.0
830,37.61,-121.99,9.0,3666.0,711.0,2341.0,703.0,4.6458,217000.0
859,37.57,-121.97,21.0,4342.0,783.0,2172.0,789.0,4.6146,247600.0
860,37.58,-121.96,15.0,3575.0,597.0,1777.0,559.0,5.7192,283500.0
861,37.58,-121.98,20.0,4126.0,1031.0,2079.0,975.0,3.6832,216900.0
...,...,...,...,...,...,...,...,...,...
20635,39.48,-121.09,25.0,1665.0,374.0,845.0,330.0,1.5603,78100.0
20636,39.49,-121.21,18.0,697.0,150.0,356.0,114.0,2.5568,77100.0
20637,39.43,-121.22,17.0,2254.0,485.0,1007.0,433.0,1.7000,92300.0
20638,39.43,-121.32,18.0,1860.0,409.0,741.0,349.0,1.8672,84700.0


In [62]:
df.isna().sum()

latitude                0
longitude               0
housing_median_age      0
total_rooms             0
total_bedrooms        157
population              0
households              0
median_income           0
median_house_value      0
dtype: int64

In [9]:
df.population.median()

1195.0

# Prepare and split the dataset

In [10]:
np.random.seed(42)

In [11]:
idx=np.arange(df.shape[0])

In [14]:
df_length=df.shape[0]
df_train_length=int(df_length*(.6))
df_val_length=int(df_length*(.2))
df_test_length=int(df_length*(.2))

In [15]:
idx

array([    0,     1,     2, ..., 15684, 15685, 15686])

In [16]:
np.random.shuffle(idx)

In [17]:

df_train=df.iloc[idx[:df_train_length]].reset_index(drop=True)

In [18]:

df_val=df.iloc[idx[df_train_length:df_train_length+df_val_length]].reset_index(drop=True)
df_test=df.iloc[idx[df_train_length+df_val_length:]].reset_index(drop=True)

In [19]:
df.median_house_value=np.log1p(df.median_house_value)

In [20]:
y=np.log1p(df.median_house_value).values

In [21]:
y_train=np.log1p(df_train.median_house_value).values
y_val=np.log1p(df_val.median_house_value).values
y_test=np.log1p(df_test.median_house_value).values


In [8]:
def prep_data(df):
    new=df.copy()
    cols=["total_rooms","total_bedrooms","population","households"]
    new=new[cols]
    new=new.fillna(0)
    return new
    

In [23]:
train_x_fillzero=prep_data(df_val)

# filling with 0

In [9]:
def train_linear_reg(X,y):
    ones=np.ones(X.shape[0])
    X=np.column_stack([ones,X])
    XTX=X.T.dot(X)
    XTX_inv=np.linalg.inv(XTX)
    w=XTX_inv.dot(X.T).dot(y)
    return w[0],w[1:]

In [25]:
w0,w=train_linear_reg(train_x_fillzero,y_val)

In [26]:
y_pred=w0+train_x_fillzero.dot(w)

In [27]:
y_pred

0       11.960420
1       11.950467
2       11.995731
3       11.797565
4       12.068567
          ...    
3132    12.031879
3133    11.950303
3134    12.488310
3135    12.294891
3136    11.982729
Length: 3137, dtype: float64

# filling with mean

In [10]:
def prep_data_mean(df):
    new=df.copy()
    cols=["total_rooms","total_bedrooms","population","households"]
    new=new[cols]
    new=new.fillna(round(df_val.total_bedrooms.mean(),2))
    return new

In [29]:
train_x_fill_mean=prep_data_mean(df_val)

In [30]:
W0,W=train_linear_reg(train_x_fill_mean,y_val)

In [31]:
Y_PRED=W0+train_x_fill_mean.dot(W)

In [32]:
Y_PRED

0       11.972583
1       11.960743
2       12.018445
3       11.785192
4       12.092388
          ...    
3132    12.037268
3133    11.954518
3134    12.021372
3135    12.341033
3136    12.003465
Length: 3137, dtype: float64

# rmse

In [11]:
def rmse(y,y_pred):
    diff=(y-y_pred)**2
    avg=diff.mean()
    return (np.sqrt(avg))

rmse with filling with mean

In [34]:
round(rmse(y_val,Y_PRED),2)

0.55

rmse with filling 0's

In [35]:
round(rmse(y_val,y_pred),2)

0.55

# regularization

In [12]:
def train_linear_reg_regularization(X,y,r):
    ones=np.ones(X.shape[0])
    X=np.column_stack([ones,X])
    XTX=X.T.dot(X)
    XTX=XTX+r*(np.eye(XTX.shape[0]))
    XTX_inv=np.linalg.inv(XTX)
    w=XTX_inv.dot(X.T).dot(y)
    return w[0],w[1:]

In [37]:
for r in [0, 0.000001, 0.0001, 0.001, 0.01, 0.1, 1, 5, 10]:
    train_x=prep_data(df_train)
    w0,w=train_linear_reg_regularization(train_x,y_train,r)
    val_x=prep_data(df_val)
    y_pred=w0+val_x.dot(w)
    print(rmse(y_pred,y_val))

0.55
0.55
0.55
0.55
0.55
0.55
0.55
0.55
0.55


In [38]:
train_x

,total_rooms,total_bedrooms,population,households
0,2935.0,523.0,1927.0,530.0
1,2426.0,426.0,1319.0,446.0
2,3653.0,568.0,1930.0,585.0
3,2668.0,609.0,1512.0,541.0
4,2479.0,541.0,1990.0,506.0
...,...,...,...,...
9407,2075.0,424.0,1312.0,396.0
9408,2005.0,392.0,1043.0,351.0
9409,1673.0,385.0,1434.0,371.0
9410,1238.0,147.0,377.0,145.0


In [39]:
val_x

,total_rooms,total_bedrooms,population,households
0,1307.0,441.0,884.0,456.0
1,1608.0,297.0,1057.0,295.0
2,1410.0,214.0,837.0,240.0
3,1116.0,328.0,1265.0,302.0
4,2409.0,436.0,1142.0,440.0
...,...,...,...,...
3132,3293.0,585.0,1678.0,530.0
3133,2610.0,597.0,1659.0,571.0
3134,2516.0,0.0,1266.0,494.0
3135,5308.0,854.0,2114.0,838.0



# shuffling

In [79]:
def shufling(seed,df):
    np.random.seed(seed)
    df_length=df.shape[0]
    df_train_length=int(df_length*(.6))
    df_val_length=int(df_length*(.2))
    df_test_length=int(df_length*(.2))
    idx=np.arange(df.shape[0])
    np.random.shuffle(idx)
    df_train=(df.iloc[idx[:df_train_length]])
    df_val=(df.iloc[idx[df_train_length:df_train_length+df_val_length]])
    df_test=(df.iloc[idx[df_train_length+df_val_length:]])
    y_train=np.log1p(df_train.median_house_value).values
    y_val=np.log1p(df_val.median_house_value).values
    y_test=np.log1p(df_test.median_house_value).values
    train_x=prep_data(df_train)
    val_x=prep_data(df_val)
    test_x=prep_data(df_test)
    W0,W=train_linear_reg(train_x,y_train)
    Y_PRED=W0+val_x.dot(W)
    
    return rmse(Y_PRED,y_val)
    
    
    

In [80]:
rmse_sum=[]
for s in [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]:
    rmse_sum.append(shufling(s,df))
    

In [81]:
round(np.std(rmse_sum),3)

0.005

In [40]:
df

,latitude,longitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
701,37.64,-121.97,32.0,1283.0,194.0,485.0,171.0,6.0574,12.973866
830,37.61,-121.99,9.0,3666.0,711.0,2341.0,703.0,4.6458,12.287657
859,37.57,-121.97,21.0,4342.0,783.0,2172.0,789.0,4.6146,12.419574
860,37.58,-121.96,15.0,3575.0,597.0,1777.0,559.0,5.7192,12.554971
861,37.58,-121.98,20.0,4126.0,1031.0,2079.0,975.0,3.6832,12.287196
...,...,...,...,...,...,...,...,...,...
20635,39.48,-121.09,25.0,1665.0,374.0,845.0,330.0,1.5603,11.265758
20636,39.49,-121.21,18.0,697.0,150.0,356.0,114.0,2.5568,11.252872
20637,39.43,-121.22,17.0,2254.0,485.0,1007.0,433.0,1.7000,11.432810
20638,39.43,-121.32,18.0,1860.0,409.0,741.0,349.0,1.8672,11.346883


# using seed value 9

In [13]:
np.random.seed(9)

In [16]:
df_length=df.shape[0]
df_train_length=int(df_length*(.6))
df_val_length=int(df_length*(.2))
df_test_length=int(df_length*(.2))

In [17]:
idx=np.arange(df.shape[0])
np.random.shuffle(idx)


In [18]:
df_train=(df.iloc[idx[:df_train_length]])
df_val=(df.iloc[idx[df_train_length:df_train_length+df_val_length]])
df_test=(df.iloc[idx[df_train_length+df_val_length:]])

In [19]:
combined_train_val=prep_data(pd.concat([df_train,df_val]).reset_index(drop=True))

In [28]:
combined_y=np.concatenate((y_train,y_val))

In [27]:
combined_train_val

,total_rooms,total_bedrooms,population,households
0,4046.0,731.0,1684.0,701.0
1,5166.0,770.0,2374.0,753.0
2,1018.0,169.0,484.0,174.0
3,1604.0,284.0,639.0,278.0
4,2744.0,541.0,1333.0,503.0
...,...,...,...,...
12544,2497.0,500.0,1357.0,456.0
12545,2069.0,312.0,889.0,299.0
12546,763.0,191.0,754.0,174.0
12547,5211.0,949.0,3025.0,948.0


In [30]:
len(combined_y)

12549

In [21]:

y_train=np.log1p(df_train.median_house_value).values
y_val=np.log1p(df_val.median_house_value).values
y_test=np.log1p(df_test.median_house_value).values

In [23]:
train_x=prep_data(df_train)
val_x=prep_data(df_val)
test_x=prep_data(df_test)

In [24]:

W0,W=train_linear_reg_regularization(combined_train_val,combined_y,r=0.001)
Y_PRED=W0+test_x.dot(W)

In [25]:
Y_PRED

17992    12.135869
6929     11.778735
16483    12.029866
12645    11.794454
13648    11.916217
           ...    
7660     11.923988
1543     12.284027
8450     11.939727
5904     11.881362
10633    12.112256
Length: 3138, dtype: float64

In [26]:
round(rmse(Y_PRED,y_test),3)

0.535